In [1]:
import pandas as pd
import random

In [2]:
minutes = range(1, 1441)
dataChoices = ['foo', 'bar', 'foobar', 'fiz', 'buzz', 'fizzbuzz']

In [3]:
# Properties the data must have for each user

# 1. Each number from N = {1, 2, 3, ... 1440} must be present
#    Solution:  For each user,
#               Check for SUM(minutes) = 1037520 

# 2. Numbers outside of N, e.g. -1, 1441, etc. are NOT allowed
#    Solution: For each user,
#              Check for MIN(minutes) = 1 AND MAX(minutes) = 1440

# 3. Unqique entry of any minute in N for a given user 
#    Solution: For each user,
#              Check for COUNT(DISTINCT(minutes) = 1440

In [4]:
# Define patients 1 and 2, as having all records conforming to spec

df1 = pd.DataFrame({'user_id'   : [1 for m in minutes],
                    'minutes'   : minutes,
                    'data'      : [random.choice(dataChoices) for _ in minutes]})

df2 = pd.DataFrame({'user_id'   : [2 for m in minutes],
                    'minutes'   : minutes,
                    'data'      : [random.choice(dataChoices) for _ in minutes]})

## Define a test function to ensure that the fake data can generate all of the possible failure modes

In [5]:
from collections import Counter

def num_distinct(arr):
    return sum(Counter(arr).values())

def test_minutes(arr):
    
    sumTest      =  ( sum(arr) == 1037520 ) # sum(1, 2, ...., 1440) = 1037520 
    boundaryTest = ( min(arr) == 1 and max(arr) == 1440 ) 
    distinctTest = ( num_distinct(arr) == 1440 )
   
    if sumTest:      print "Summation test:  PASS"
    else:            print "Summation test:  FAIL"
        
    if boundaryTest: print "Boundary test:   PASS"
    else:            print "Boundary test:   FAIL"    
    
    if distinctTest: print "Uniqueness test: PASS"
    else:            print "Uniqueness test: FAIL"
    
    return

In [6]:
# Define some malformed minutes data, there should be a max of 2^3 - 1 non-passing combinations:

# Sum test          : FAIL 
# Boundary test     : FAIL
# Unique count test : FAIL
user3Min = [-1] + minutes
test_minutes(user3Min)

Summation test:  FAIL
Boundary test:   FAIL
Uniqueness test: FAIL


In [7]:
# Sum test          : FAIL
# Boundary test     : FAIL
# Unique count test : PASS
user4Min = [x for x in range(1, 1441) if x != 42] + [1441] 
test_minutes(user4Min)

Summation test:  FAIL
Boundary test:   FAIL
Uniqueness test: PASS


In [8]:
# CANNOT OCCUR
# Sum test          : FAIL
# Boundary test     : PASS
# Unique count test : PASS

In [9]:
# Sum test          : FAIL 
# Boundary test     : PASS
# Unique count test : FAIL
user5Min = [1, 2, 3] + range(5, 1441)
test_minutes(user5Min)

Summation test:  FAIL
Boundary test:   PASS
Uniqueness test: FAIL


In [10]:
# Sum test          : PASS  
# Boundary test     : PASS
# Unique count test : FAIL
user6Min = [1, 2, 2, 2, 3] + range(5, 1441)
test_minutes(user6Min)

Summation test:  PASS
Boundary test:   PASS
Uniqueness test: FAIL


In [11]:
# Sum test          : PASS 
# Boundary test     : FAIL
# Unique count test : FAIL
user7Min = [-3, 1, 2, 3, 3] + range(4, 1441)
test_minutes(user7Min)

Summation test:  PASS
Boundary test:   FAIL
Uniqueness test: FAIL


In [12]:
# Sum test          : PASS 
# Boundary test     : FAIL
# Unique count test : PASS
user8Min = range(-1440/2, 0) + range(2, 1440/2 + 1) + [1037521]
test_minutes(user8Min)

Summation test:  PASS
Boundary test:   FAIL
Uniqueness test: PASS


In [13]:
df3 = pd.DataFrame({'user_id'   : [3 for m in user3Min],
                    'minutes'   : user3Min,
                    'data'      : [random.choice(dataChoices) for _ in user3Min]})

df4 = pd.DataFrame({'user_id'   : [4 for m in user4Min],
                    'minutes'   : user4Min,
                    'data'      : [random.choice(dataChoices) for _ in user4Min]})

df5 = pd.DataFrame({'user_id'   : [5 for m in user5Min],
                    'minutes'   : user5Min,
                    'data'      : [random.choice(dataChoices) for _ in user5Min]})

df6 = pd.DataFrame({'user_id'   : [6 for m in user6Min],
                    'minutes'   : user6Min,
                    'data'      : [random.choice(dataChoices) for _ in user6Min]})

df7 = pd.DataFrame({'user_id'   : [7 for m in user7Min],
                    'minutes'   : user7Min,
                    'data'      : [random.choice(dataChoices) for _ in user7Min]})

df8 = pd.DataFrame({'user_id'   : [8 for m in user8Min],
                    'minutes'   : user8Min,
                    'data'      : [random.choice(dataChoices) for _ in user8Min]})

In [14]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8])

In [15]:
from pandasql import sqldf

In [16]:
pysqldf = lambda q: sqldf(q, globals())

In [17]:
q = """

SELECT *

FROM

    (SELECT
        user_id,
        MIN(minutes) as min_minute,
        MAX(minutes) as max_minute,
        SUM(minutes) as sum_minutes,
        COUNT(DISTINCT(minutes)) as distinct_minutes

    FROM df

    GROUP BY user_id) a

WHERE
    min_minute       = 1
AND max_minute       = 1440
AND sum_minutes      = 1037520
AND distinct_minutes = 1440
    

"""

In [18]:
output = pysqldf(q)

## Inspecting the output shows what we expect -- only users 1 and 2 are returned because all other users have been filtered out due to one of the following:
1. Malformed data
2. Missing data
3. Duplicate data

In [19]:
output

,user_id,min_minute,max_minute,sum_minutes,distinct_minutes
0,1,1,1440,1037520,1440
1,2,1,1440,1037520,1440
